<a href="https://colab.research.google.com/github/ggierlik/dw_matrix_road_sign/blob/master/day5b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [13]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import os
import datetime

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:] #obcina pierwszy element
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(X_train, y_train, 
            batch_size=params_fit.get('batch_size', 128), #rozmiar paczki wejściowej
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks=[tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_model(params):
    return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),
    
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_danse_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_danse_block_two']),
        
    Dense(num_classes, activation='softmax'),
  ])
    

In [12]:
model = get_cnn_v5(input_shape, num_classes) # zmienily sie kanaly
model_trained = train_model(model, X_train, y_train, params_fit={})
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 6s 24ms/step - loss: 2.5977 - accuracy: 0.2804 - val_loss: 1.2784 - val_accuracy: 0.5926
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 1.0953 - accuracy: 0.6534 - val_loss: 0.3812 - val_accuracy: 0.8880
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.5356 - accuracy: 0.8297 - val_loss: 0.1598 - val_accuracy: 0.9500
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3400 - accuracy: 0.8919 - val_loss: 0.0982 - val_accuracy: 0.9677
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2743 - accuracy: 0.9157 - val_loss: 0.0594 - val_accuracy: 0.9826


0.9512471655328798

In [0]:
def func_obj(params):
  model = get_model(params) # zmienily sie kanaly
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(X_train,
            y_train, 
            batch_size=int(params.get('batch_size', 128)), #rozmiar paczki wejściowej
            epochs=5,
            verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model} # minus aby dążyć do najmniejszej wartości -1 (1=100% najlepsza wartość)

In [35]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_danse_block_one': hp.uniform('dropout_danse_block_one', 0.3, 0.7),
    'dropout_danse_block_two': hp.uniform('dropout_danse_block_two', 0.3, 0.7),
}
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.43286620556623245, 'dropout_cnn_block_three': 0.4601240108831398, 'dropout_cnn_block_two': 0.4371322354303627, 'dropout_danse_block_one': 0.44524819053493236, 'dropout_danse_block_two': 0.6674435228288138}
accuracy=0.9263038635253906
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.34393958150563286, 'dropout_cnn_block_three': 0.40903760533658196, 'dropout_cnn_block_two': 0.34728045821634485, 'dropout_danse_block_one': 0.6864814222242053, 'dropout_danse_block_two': 0.4818976785293213}
accuracy=0.9399092793464661
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.33345425910584997, 'dropout_cnn_block_three': 0.3524430536513609, 'dropout_cnn_block_two': 0.459843664373528, 'dropout_danse_block_one': 0.4227500367384044, 'dropout_danse_block_two': 0.5318266410401256}
accuracy=0.9294784665107727
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.42959897793494306, 'dropout_cnn_block_three': 0.3991001610139031, 'dropout_cnn_block_two': 0.3783074554